In [1]:
from typing import List, Optional
import random
import time
import os

import requests
import ipyplot

# Path to Nebius API key, see instruction on how to get one: https://docs.nebius.com/studio/api/authentication
# Alternatively you can just put it in the notebook for running samples easier:
# TOKEN = *You Nebius API key*
with open(os.path.expanduser("~/.secrets/nebius-openai-api-key")) as f:
  TOKEN = f.read().strip()

def gen_image(model: str, loras: Optional[List[dict]], prompt: str, width: int, height: int, num_inference_steps: int):
  resp = requests.post(
    url="https://api.studio.nebius.com/v1/images/generations",
    headers={"Authorization": f"Bearer {TOKEN}"},
    json={
      "model": model,
      # NEW optional (backward compatible) field
      # [{url, scale}, ...]
      "loras": loras,
      "prompt": prompt,
      "width": width,
      "height": height,
      "num_inference_steps": num_inference_steps,
    },
  )
  if resp.status_code != 200:
    print(resp.text)
    resp.raise_for_status()

  image_url = resp.json()['data'][0]['url']
  content = None
  for i in range(3):
    resp = requests.get(image_url)
    if resp.status_code == 200:
      content = resp.content
      break
    if resp.status_code != 200:
      # BUG: returns 404
      print("No photo:", resp.text)
      time.sleep(1)
      continue
      # resp.raise_for_status()

  if content is None:
    resp.raise_for_status()

  os.makedirs("images", exist_ok=True)
  name = f"images/{''.join([random.choice('abcde') for _ in range(10)])}.webp"
  with open(name, "wb") as f:
    f.write(content)
  return name

def show_3_pics(**kwargs):
  measure = kwargs.get('measure', False)
  if 'measure' in kwargs:
    del kwargs['measure']

  images_list = []
  for _ in range(3):
    before = time.time()
    images_list.append(gen_image(**kwargs))
    if measure:
      print(f"Image generated in {round(time.time() - before)}sec")
  ipyplot.plot_images(images_list, max_images=3, img_width=256)

# No LoRA

In [2]:
show_3_pics(
    model="black-forest-labs/flux-dev",
    loras=None,
    prompt="A fancy cat in sunglasses, Studio Ghibli Dark Fairytale, ArsMJStyle, impressionism",
    width=256,
    height=256,
    num_inference_steps=26,
)

# Let's add 1 LoRA

## Dark Ghibli LoRA

In [3]:
show_3_pics(
    model="black-forest-labs/flux-dev",
    loras=[
      {"url": "https://civitai.com/api/download/models/1524423?type=Model&format=SafeTensor", "scale": 1},
    ],
    prompt="A fancy cat in sunglasses, Studio Ghibli Dark Fairytale, ArsMJStyle, impressionism",
    width=256,
    height=256,
    num_inference_steps=26,
)

## Impressionism LoRA

In [4]:
show_3_pics(
    model="black-forest-labs/flux-dev",
    loras=[
      {"url": "https://civitai.com/api/download/models/755598?type=Model&format=SafeTensor", "scale": 0.8},
    ],
    prompt="A fancy cat in sunglasses, Studio Ghibli Dark Fairytale, ArsMJStyle, impressionism",
    width=256,
    height=256,
    num_inference_steps=26,
)

# Let's use both

In [5]:
show_3_pics(
    model="black-forest-labs/flux-dev",
    loras=[
      {"url": "https://civitai.com/api/download/models/1524423?type=Model&format=SafeTensor", "scale": 0.7},
      {"url": "https://civitai.com/api/download/models/755598?type=Model&format=SafeTensor", "scale": 0.7},
    ],
    prompt="A fancy cat in sunglasses, Studio Ghibli Dark Fairytale, ArsMJStyle, impressionism",
    width=256,
    height=256,
    num_inference_steps=26,
)

# Other models & auth

## No LoRA

In [6]:
show_3_pics(
    model="black-forest-labs/flux-schnell",
    loras=None,
    prompt="xuer Asgard City,Diesel punk, diesel punk style,Realistic，Hyper-realistic，motion blur,going really fast masterpiece, deep focus, exaggerated Ghost in the Shell scenes and girl In a sci-fi mechanical chariot, in the style of cyberpunk imagery, in the style of hyper-realistic sci-fi, 32k uhd, marc simonetti, realistic and hyper-detailed renderings, snailcore, artgerm, neo-futuristic grand cybernetwork buildings, patchwork, exquisite details, complex and reasonable structures, bright, daytime, bustling cybercity and streets,full-line projection screens, special effects, laser rain, fluorescence, Asgard City, patchwork,  neon lights, Regular, Special Effects, Holographic Network, Exaggerated Facade, Walker, cinematic photo This is a poster about the Asgard City，It has a realistic picture effect，The picture is messy and extremely rich in details，(The theme text of the poster is: 'XUER':1.2)，One line apart,(the subtitle below reads: 'Asgard City':1.2), golden hour,unconventional supreme masterpiece,masterful details,temperate atmosphere,with a high-end texture,in the style of fashion photography,magazine cover,Dynamic Angle,Dynamic posture,Perspective, unconventional supreme masterpiece,masterful details,see-through,light particles,realistic，Silk, embroidery,Rust,Blue eyes",
    width=256,
    height=256,
    num_inference_steps=16,
)

## LoRA under authwall
https://civitai.com/models/329635?modelVersionId=899864

In [7]:
show_3_pics(
    model="black-forest-labs/flux-schnell",
    loras=[
      {"url": "https://civitai.com/models/329635?modelVersionId=899864", "scale": 1.0},
    ],
    prompt="xuer Asgard City,Diesel punk, diesel punk style,Realistic，Hyper-realistic，motion blur,going really fast masterpiece, deep focus, exaggerated Ghost in the Shell scenes and girl In a sci-fi mechanical chariot, in the style of cyberpunk imagery, in the style of hyper-realistic sci-fi, 32k uhd, marc simonetti, realistic and hyper-detailed renderings, snailcore, artgerm, neo-futuristic grand cybernetwork buildings, patchwork, exquisite details, complex and reasonable structures, bright, daytime, bustling cybercity and streets,full-line projection screens, special effects, laser rain, fluorescence, Asgard City, patchwork,  neon lights, Regular, Special Effects, Holographic Network, Exaggerated Facade, Walker, cinematic photo This is a poster about the Asgard City，It has a realistic picture effect，The picture is messy and extremely rich in details，(The theme text of the poster is: 'XUER':1.2)，One line apart,(the subtitle below reads: 'Asgard City':1.2), golden hour,unconventional supreme masterpiece,masterful details,temperate atmosphere,with a high-end texture,in the style of fashion photography,magazine cover,Dynamic Angle,Dynamic posture,Perspective, unconventional supreme masterpiece,masterful details,see-through,light particles,realistic，Silk, embroidery,Rust,Blue eyes",
    width=256,
    height=256,
    num_inference_steps=16,
)

# LoRA validation and SDXL

Only civitai is allowed by now

In [8]:
try:
    show_3_pics(
        model="black-forest-labs/flux-dev",
        loras=[
          {"url": "https://civitai.com/api/download/models/1524423?type=Model&format=SafeTensor", "scale": 0.7},
          {"url": "https://example.com/lora.safetensors", "scale": 0.7},
        ],
        prompt="A fancy cat in sunglasses, Studio Ghibli Dark Fairytale, ArsMJStyle, impressionism",
        width=256,
        height=256,
        num_inference_steps=26,
    )
except Exception as e:
    print(e)

{"detail":"LoRA validation error (https://example.com/lora.safetensors): Unexpected URL host: example.com"}
400 Client Error: Bad Request for url: https://api.studio.nebius.com/v1/images/generations


LoRAs can be < 1GiB, I validate it before downloading

In [9]:
try:
    show_3_pics(
        model="black-forest-labs/flux-dev",
        loras=[
          {"url": "https://civitai.com/api/download/models/1524423?type=Model&format=SafeTensor", "scale": 0.7},
          {"url": "https://civitai.com/models/251417/midjourney-mimic?modelVersionId=283697", "scale": 0.7},
        ],
        prompt="A fancy cat in sunglasses, Studio Ghibli Dark Fairytale, ArsMJStyle, impressionism",
        width=256,
        height=256,
        num_inference_steps=26,
    )
except Exception as e:
    print(e)

{"detail":"LoRA validation error (https://civitai.com/models/251417/midjourney-mimic?modelVersionId=283697): LoRA cannot be larger than 1073741824bytes"}
400 Client Error: Bad Request for url: https://api.studio.nebius.com/v1/images/generations


civitai.green now works, also caching of LoRA works

In [10]:
show_3_pics(
    model="black-forest-labs/flux-dev",
    loras=[
      {"url": "https://civitai.green/models/1381703/80s-vintage-photography-schnell-flux-lora?modelVersionId=1590102", "scale": 1.0},
    ],
    prompt="Cat in vintage glasses, old film quality, vntg80_photo",
    width=256,
    height=256,
    num_inference_steps=26,
    measure=True,
)

Image generated in 6sec
Image generated in 5sec
Image generated in 5sec


## You can also use SDXL 1.0

Without LoRA

In [11]:
show_3_pics(
    model="stability-ai/sdxl",
    loras=None,
    prompt="a Delorean on synthwave city <lora:pixelbuildings128-v2:1>",
    width=1024,
    height=1024,
    num_inference_steps=64,
    measure=True,
)

Image generated in 8sec
Image generated in 8sec
Image generated in 13sec


With LoRA

In [12]:
show_3_pics(
    model="stability-ai/sdxl",
    loras=[
      {"url": "https://civitai.com/models/120096/pixel-art-xl?modelVersionId=135931", "scale": 1.0},
    ],
    prompt="a Delorean on synthwave city <lora:pixelbuildings128-v2:1>",
    width=1024,
    height=1024,
    num_inference_steps=64,
    measure=True,
)

Image generated in 19sec
Image generated in 11sec
Image generated in 11sec


Without LoRA

In [13]:
show_3_pics(
    model="stability-ai/sdxl",
    loras=None,
    prompt="dwarf warrior, long beard, heavily armored, shield, fantasy, epic, in the snow,  <lora:EpicF4nta5yXL:0.8>",
    width=1024,
    height=1024,
    num_inference_steps=64,
    measure=True,
)

Image generated in 7sec
Image generated in 7sec
Image generated in 7sec


Some other LoRA

In [14]:
show_3_pics(
    model="stability-ai/sdxl",
    loras=[
      {"url": "https://civitai.com/models/470073/popyays-epic-fantasy-style-or-pony-and-sdxl?modelVersionId=560995", "scale": 0.8},
    ],
    prompt="dwarf warrior, long beard, heavily armored, shield, fantasy, epic, in the snow,  <lora:EpicF4nta5yXL:0.8>",
    width=1024,
    height=1024,
    num_inference_steps=64,
    measure=True,
)

Image generated in 10sec
Image generated in 10sec
Image generated in 10sec
